# BIRD SOUND CLASSIFICATION - COMPLETE ML PIPELINE
### African Leadership University - Machine Learning Pipeline Assignment

# Date: November 2024
####  ML pipeline for audio classification


#### 🐦 Bird Sound Classification - Complete ML Pipeline

## Table of Contents
1. Exploratory Data Analysis 
2. Audio Feature Visualization
3. Data Preprocessing and Feature Extraction
4. Feature Analysis and Interpretations 
5. Train-Test Split Strategy
6. Model Architecture and Training
7. Model Evaluation and Metrics 
8. Error Analysis
9. Model Deployment Preparation
10. Conclusions and Future Work

#### Project Overview
This notebook implements a complete machine learning pipeline for classifying 
64 bird species based on audio recordings. We extract 95 acoustic features 
and train a deep neural network to achieve high accuracy classification.

**Key Features:**
- 64 bird species classification
- 95 audio features 
- Deep Neural Network 
- Production-ready model deployment


In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import json
from datetime import datetime
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                            precision_recall_fscore_support, f1_score,
                            precision_score, recall_score)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

print(f"TensorFlow: {tf.__version__} | GPU: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
#  LOAD DATASET

def load_dataset(csv_path):
    """Load dataset from CSV and extract temporal features"""
    df = pd.read_csv(csv_path)
    df['year'] = df['Date'].apply(lambda x: str(x).split('-')[0])
    df['month'] = df['Date'].apply(lambda x: str(x).split('-')[1])
    df['day_of_month'] = df['Date'].apply(lambda x: str(x).split('-')[2])
    
    print("DATASET LOADED")
    print(f"Shape: {df.shape} | Species: {df['common_name'].nunique()}")
    return df

# UPDATE WITH YOUR CSV PATH
CSV_PATH = r"C:\Users\PC\Desktop\assignment---MLOP\audio_processing_fe_bird_sounds\data\raw\Birds Voice.csv"
train_csv = load_dataset(CSV_PATH)

print(train_csv.head())
print(f"\nColumns: {train_csv.columns.tolist()}")

In [ ]:
#  AUDIO PROCESSING FUNCTIONS

def load_and_trim_audio(file_path, sr=22050, top_db=23):
    """Load and trim silence from audio file"""
    try:
        data, sample_rate = librosa.load(file_path, sr=sr)
        trimmed_data, _ = librosa.effects.trim(data, top_db=top_db)
        return trimmed_data, sample_rate
    except Exception as e:
        return None, None

def extract_features(audio_data, sr=22050):
    """Extract 95 audio features WITHOUT fitting any scalers"""
    if audio_data is None or len(audio_data) == 0:
        return None
    try:
        features = []
        
        # MFCC 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=20)
        features.extend(np.mean(mfccs, axis=1))
        features.extend(np.std(mfccs, axis=1))
        features.extend(np.max(mfccs, axis=1))
        features.extend(np.min(mfccs, axis=1))
        
        # Spectral features
        sc = librosa.feature.spectral_centroid(y=audio_data, sr=sr)
        features.extend([np.mean(sc), np.std(sc), np.max(sc)])
        
        sr_off = librosa.feature.spectral_rolloff(y=audio_data, sr=sr)
        features.extend([np.mean(sr_off), np.std(sr_off)])
        
        zcr = librosa.feature.zero_crossing_rate(audio_data)
        features.extend([np.mean(zcr), np.std(zcr)])
        
        chroma = librosa.feature.chroma_stft(y=audio_data, sr=sr)
        features.extend([np.mean(chroma), np.std(chroma)])
        
        mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sr, n_mels=128)
        mel_db = librosa.power_to_db(mel_spec, ref=np.max)
        features.extend([np.mean(mel_db), np.std(mel_db), np.max(mel_db)])
        
        sbw = librosa.feature.spectral_bandwidth(y=audio_data, sr=sr)
        features.extend([np.mean(sbw), np.std(sbw)])
        
        rms = librosa.feature.rms(y=audio_data)
        features.append(np.mean(rms))
        
        return np.array(features)
    except Exception as e:
        print(f"Feature extraction error: {e}")
        return None

In [ ]:
# PROCESS AUDIO FILES

AUDIO_BASE_PATH = r"C:\Users\PC\Desktop\assignment---MLOP\audio_processing_fe_bird_sounds\data\Voice of Birds\Voice of Birds"

def process_dataset_with_unique_samples(df, audio_base_path, samples_per_species=20, min_samples_per_class=15):
    
    features_list = []
    labels_list = []
    file_paths_list = []
    
    species_counts = df['common_name'].value_counts()
    valid_species = species_counts[species_counts >= min_samples_per_class].index
    
    print(f"Processing {len(valid_species)} species with >= {min_samples_per_class} samples each")
    
    available_folders = os.listdir(audio_base_path)
    
    def find_species_folder(common_name):
        for folder in available_folders:
            folder_clean = folder.lower().replace('_', ' ').replace('sound', '').strip()
            if common_name.lower() in folder_clean or folder_clean in common_name.lower():
                return folder
        return None
    
    for species in tqdm(valid_species, desc="Processing species"):
        species_folder = find_species_folder(species)
        if not species_folder:
            continue
        
        folder_path = os.path.join(audio_base_path, species_folder)
        mp3_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.mp3')]
        
        if len(mp3_files) == 0:
            continue
        
        np.random.shuffle(mp3_files)
        files_to_process = mp3_files[:samples_per_species]
        
        for mp3_file in files_to_process:
            try:
                file_path = os.path.join(folder_path, mp3_file)
                audio_data, sr = load_and_trim_audio(file_path)
                
                if audio_data is None or len(audio_data) < sr * 0.5:
                    continue
                
                features = extract_features(audio_data, sr)
                if features is None or np.isnan(features).any():
                    continue
                
                features_list.append(features)
                labels_list.append(species)
                file_paths_list.append(file_path)
                
            except Exception as e:
                continue
    
    print(f"\nProcessed: {len(features_list)} samples from {len(np.unique(labels_list))} species")
    return np.array(features_list), np.array(labels_list), file_paths_list

X, y, file_paths = process_dataset_with_unique_samples(
    train_csv, AUDIO_BASE_PATH, samples_per_species=25, min_samples_per_class=15
)

print(f"Raw Features: {X.shape} | Labels: {y.shape}")
print(f"Unique species: {len(np.unique(y))}")

In [ ]:
#  TRAIN and TEST SPLIT & SCALING 

# CRITICAL: Encode and split BEFORE scaling
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_encoded, test_size=0.2, random_state=RANDOM_SEED, stratify=y_encoded
)

print(f"Data split BEFORE scaling")
print(f"  Train: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Test: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

y_train = to_categorical(y_train_encoded)
y_test = to_categorical(y_test_encoded)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nScaling applied correctly")
print(f"  Train mean: {X_train_scaled.mean():.6f}, std: {X_train_scaled.std():.6f}")
print(f"  Test mean: {X_test_scaled.mean():.6f}, std: {X_test_scaled.std():.6f}")

In [ ]:
#  SAVE DATA

base_path = r"C:\Users\PC\Desktop\assignment---MLOP\audio_processing_fe_bird_sounds\data"
train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

np.save(os.path.join(train_path, "X_train.npy"), X_train_scaled)
np.save(os.path.join(train_path, "y_train.npy"), y_train_encoded)
np.save(os.path.join(test_path, "X_test.npy"), X_test_scaled)
np.save(os.path.join(test_path, "y_test.npy"), y_test_encoded)

print("Train and test sets saved successfully!")

In [ ]:
#  BUILD MODEL ARCHITECTURE


def create_optimized_model(input_dim, num_classes):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
    )
    return model

model = create_optimized_model(X_train_scaled.shape[1], y_train.shape[1])
print("MODEL ARCHITECTURE")
model.summary()

In [ ]:
# TRAIN MODEL


os.makedirs('models', exist_ok=True)

callbacks = [
    ModelCheckpoint('models/best_model.h5', monitor='val_accuracy', 
                   save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, 
                     min_lr=0.00001, verbose=1)
]

print("TRAINING MODEL")

history = model.fit(
    X_train_scaled, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

print("Training completed!")

In [ ]:
#  PLOT TRAINING HISTORY


fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Training History', fontsize=20, fontweight='bold')

axes[0, 0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0, 0].plot(history.history['val_accuracy'], label='Val', linewidth=2)
axes[0, 0].set_title('Accuracy'); axes[0, 0].legend(); axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history.history['loss'], label='Train', linewidth=2)
axes[0, 1].plot(history.history['val_loss'], label='Val', linewidth=2)
axes[0, 1].set_title('Loss'); axes[0, 1].legend(); axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(history.history['precision'], label='Train', linewidth=2)
axes[1, 0].plot(history.history['val_precision'], label='Val', linewidth=2)
axes[1, 0].set_title('Precision'); axes[1, 0].legend(); axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(history.history['recall'], label='Train', linewidth=2)
axes[1, 1].plot(history.history['val_recall'], label='Val', linewidth=2)
axes[1, 1].set_title('Recall'); axes[1, 1].legend(); axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300)
plt.show()


In [ ]:
# EVALUATE MODEL


print("MODEL EVALUATION ON TEST SET")

y_pred_probs = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
test_loss = model.evaluate(X_test_scaled, y_test, verbose=0)

print("PERFORMANCE METRICS (4+ Required)")
print(f"1. ACCURACY:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"2. PRECISION: {precision:.4f} ({precision*100:.2f}%)")
print(f"3. RECALL:    {recall:.4f} ({recall*100:.2f}%)")
print(f"4. F1-SCORE:  {f1:.4f} ({f1*100:.2f}%)")
print(f"5. TEST LOSS: {test_loss[0]:.4f}")

if accuracy > 0.95:
    print("\nWARNING: Very high accuracy - check for overfitting")
else:
    print("\n✓ Realistic accuracy indicates proper train/test separation")

print("\nDETAILED CLASSIFICATION REPORT")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_, zero_division=0))

In [ ]:
#  CONFUSION MATRIX

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(20, 20))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300)
plt.show()